# Testing the unified reynolds equation solver

In [1]:
from collections import OrderedDict

In [2]:
a = {2:2,3:3,4:4}

In [7]:
OrderedDict([1,2,3])

TypeError: 'int' object is not iterable

In [6]:
for key, value in OrderedDict(a).items():
    print(f'{key}:{value}')

2:2
3:3
4:4


In [8]:
[].items()

AttributeError: 'list' object has no attribute 'items'

In [9]:
function

NameError: name 'function' is not defined

In [10]:
class Constant:
    def __init__(self, value: float):
        self.value = value
    
    def __getitem__(self, item):
        return self.value

In [17]:
import numpy as np
np.ones((5,5))


array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [18]:
from scipy import __all__ as var

In [19]:
var

['test',
 'ModuleDeprecationWarning',
 'VisibleDeprecationWarning',
 '__version__',
 'show_config',
 'char',
 'rec',
 'memmap',
 'newaxis',
 'ndarray',
 'flatiter',
 'nditer',
 'nested_iters',
 'ufunc',
 'arange',
 'array',
 'zeros',
 'count_nonzero',
 'empty',
 'broadcast',
 'dtype',
 'fromstring',
 'fromfile',
 'frombuffer',
 'int_asbuffer',
 'where',
 'argwhere',
 'copyto',
 'concatenate',
 'fastCopyAndTranspose',
 'lexsort',
 'set_numeric_ops',
 'can_cast',
 'promote_types',
 'min_scalar_type',
 'result_type',
 'isfortran',
 'empty_like',
 'zeros_like',
 'ones_like',
 'correlate',
 'convolve',
 'inner',
 'dot',
 'outer',
 'vdot',
 'roll',
 'rollaxis',
 'moveaxis',
 'cross',
 'tensordot',
 'little_endian',
 'fromiter',
 'array_equal',
 'array_equiv',
 'indices',
 'fromfunction',
 'isclose',
 'load',
 'loads',
 'isscalar',
 'binary_repr',
 'base_repr',
 'ones',
 'identity',
 'allclose',
 'compare_chararrays',
 'putmask',
 'flatnonzero',
 'Inf',
 'inf',
 'infty',
 'Infinity',
 'nan',


In [20]:
def inner(a,s,d,**kwargs):
    return None

In [21]:
import inspect

In [23]:
a = inspect.getfullargspec(inner)

In [30]:
a

FullArgSpec(args=['a', 's', 'd'], varargs=None, varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [25]:
a.args

['a', 's', 'd']

In [28]:
assert 1==2, 'NIONONONONON'


AssertionError: NIONONONONON

In [31]:
a = {1,2,3}
b = a.update({4,5,6})

In [32]:
b

In [33]:
a

{1, 2, 3, 4, 5, 6}

In [36]:
a = slice(1,4)

In [37]:
isinstance(a, slice)

True

In [46]:
a = (slice(2,3), )
b = np.ones((5,5))

In [47]:
b[a]

array([[1., 1., 1., 1., 1.]])

In [45]:
tuple(None)

TypeError: 'NoneType' object is not iterable

In [49]:
inspect.getfullargspec(str.__len__)

FullArgSpec(args=['self'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [50]:
a = {1,2,3}
b = [5,6,7]

In [51]:
a.update(b)

In [52]:
a

{1, 2, 3, 5, 6, 7}